In [1]:
# this code is generated by the Domino Code Assist toolbar button
import domino_code_assist as dca
dca.init()

Cannot show ipywidgets in text

In [3]:
! pip install pypdf

  Obtaining dependency information for pypdf from https://files.pythonhosted.org/packages/bf/53/8840f93c5dcd108c02cac7343e194f9dc5d15ade6200ccc661ab4e1352b5/pypdf-3.16.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 6.7 MB/s eta 0:00:00a 0:00:01
DEPRECATION: feast 0.20.2 has a non-standard dependency specifier PyYAML>=5.4.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of feast or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: feast 0.20.2 has a non-standard dependency specifier tenacity>=7.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of feast or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues

In [11]:
# Load the libraries that are needed
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

import os
import random

In [20]:
# Load the document that you need to parse, please change the location to where the pdf resides
# loader = UnstructuredPDFLoader("/mnt/code/Select_Global_Value_Fund.pdf")
# data = loader.load()

loader = PyPDFLoader("/mnt/code/Select_Global_Value_Fund.pdf")
texts = loader.load_and_split(RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0))

In [21]:
print(f"There are {len(texts)} pages in the document")

There are 71 pages in the document


In [22]:
# Pick a sample page
print(texts[random.randint(0, len(texts))])

page_content="Vanguard Advice Select Global Value Fund is registered under the Investment Company Act of\n1940 as an open-end investment company, or mutual fund. The fund invests in securities offoreign issuers, which may subject it to investment risks not normally associated with investing in\nsecurities of U.S. corporations. Significant market disruptions, such as those caused by pandemics\n(e.g., COVID-19 pandemic), natural or environmental disasters, war (e.g., Russia's invasion ofUkraine), acts of terrorism, or other events, can adversely affect local and global markets andnormal market operations. Any such disruptions could have an adverse impact on the value of thefund's investments and fund performance.\nA.The following significant accounting policies conform to generally accepted accounting\nprinciples for U.S. investment companies. The fund consistently follows such policies in preparingits financial statements." metadata={'source': '/mnt/code/Select_Global_Value_Fund.pdf', '

In [23]:
#Create embeddings of your documents to get ready for semantic search

from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
import pinecone
import pickle


/opt/conda/lib/python3.9/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [24]:
# Read your OpenAI key from the environment

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY') 
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [25]:
# Index and store the embeddings locally in a pickle file
store = FAISS.from_texts([t.page_content for t in texts], embeddings)
with open("faiss_store.pkl", "wb") as f:
    pickle.dump(store, f)


In [26]:
# Query the docs to get your answer back
from langchain.llms import OpenAI

In [34]:
# Load the embeddings from the pickle file; change the location if needed
if 'store' not in locals() or store is None:
    with open("faiss_store.pkl", "rb") as f:
        store = pickle.load(f)
        
qa = RetrievalQA.from_chain_type(llm=OpenAI(model_name='gpt-3.5-turbo', temperature=0, openai_api_key=OPENAI_API_KEY), chain_type="stuff", retriever=store.as_retriever())

query = "Which countries does the fund invest in?"
docs = store.similarity_search(query)
qa({"input_documents": docs, "query": query}, return_only_outputs=True)



{'result': 'The fund invests in the United States, Japan, United Kingdom, China, France, Spain, Canada, Ireland, Hong Kong, South Korea, India, Sweden, and other countries.'}